In [5]:
import numpy as np 
import cv2
import tensorflow as tf  
from tensorflow.keras.utils import to_categorical
from keras import models, layers
from keras import backend as K
import glob
import os

In [6]:
#Definition of lightweight block (lw_conv_block)

def lw_conv_block(inputs, filter_size, filter_num, dropout, batch_norm=True):

    conv = layers.Conv2D(filter_num, (1, 1), padding='same')(inputs)
    conv = layers.Activation('relu')(conv)

    #left
    squeeze_conv = layers.Conv2D(filter_num,(1, 1), padding='same')(conv)

    # Batch normalization operation
    if batch_norm is True:
        squeeze_conv = layers.BatchNormalization(axis=3)(squeeze_conv)

    squeeze_conv = layers.Activation("relu")(squeeze_conv)

    #middle
    channel_conv = layers.DepthwiseConv2D(kernel_size=(filter_size, filter_size), strides=(1, 1), padding='same', depth_multiplier=2)(conv)
    point_conv = layers.Conv2D(2*filter_num, (1, 1), padding='same')(channel_conv)
    
    # Batch normalization operation
    if batch_norm is True:
        separa_conv = layers.BatchNormalization(axis=3)(point_conv)
    else:
        separa_conv = point_conv
        
    separa_conv = layers.Activation("relu")(separa_conv)

    #right
    expand_conv = layers.Conv2D(filter_num,(filter_size, filter_size), padding='same')(conv)
    
    # Batch normalization operation
    if batch_norm is True:
        expand_conv = layers.BatchNormalization(axis=3)(expand_conv)

    expand_conv = layers.Activation("relu")(expand_conv)

    #concatenate
    lw_conv = layers.concatenate([squeeze_conv, separa_conv, expand_conv], axis=3)
    
    # Dropout operation
    if dropout > 0:
        lw_conv = layers.Dropout(dropout)(lw_conv)

    return lw_conv

In [7]:
#Definition of convolution block

def stand_conv_block(inputs, filter_size, filter_num, dropout, batch_norm=True):

    conv = layers.Conv2D(filter_num, (filter_size, filter_size), padding='same')(inputs)
    
    # Batch normalization operation
    if batch_norm is True:

        conv = layers.BatchNormalization(axis=3)(conv)

    conv_output = layers.Activation('relu')(conv)

    # Dropout operation
    if dropout > 0:
        
        conv_output = layers.Dropout(dropout)(conv_output)

    return conv_output

In [8]:
#Definition of convolution block

def stand_conv_block(inputs, filter_size, filter_num, dropout, batch_norm=True):

    conv = layers.Conv2D(filter_num, (filter_size, filter_size), padding='same')(inputs)
    
    # Batch normalization operation
    if batch_norm is True:

        conv = layers.BatchNormalization(axis=3)(conv)

    conv_output = layers.Activation('relu')(conv)

    # Dropout operation
    if dropout > 0:
        
        conv_output = layers.Dropout(dropout)(conv_output)

    return conv_output

In [9]:
#Definition of attention block

def attention_block(x, gating, size):

# Convert gating single using (1,1) convolutions
    phi_g = layers.Conv2D(size, (1, 1), padding='same')(gating) 

# Convert x single to the same shape as the gating signal
    theta_x = layers.Conv2D(size, (1, 1), padding='same')(x) 

# Adding phi_g, theta_x together, activated by relu
    concat_xg = layers.add([phi_g, theta_x])
    act_xg = layers.Activation('relu')(concat_xg)

# Conduct ψ operation on act_xg, then activated by sigmoid
    psi = layers.Conv2D(1, (1, 1), padding='same')(act_xg)
    sigmoid_xg = layers.Activation('sigmoid')(psi)

# Conduct multiply operation on [upsample_psi, x]
    result = layers.multiply([sigmoid_xg, x])
    result_bn = layers.BatchNormalization()(result)
    
    return result_bn

In [10]:
#Definition of attention block

def attention_block(x, gating, size):

# Convert gating single using (1,1) convolutions
    phi_g = layers.Conv2D(size, (1, 1), padding='same')(gating) 

# Convert x single to the same shape as the gating signal
    theta_x = layers.Conv2D(size, (1, 1), padding='same')(x) 

# Adding phi_g, theta_x together, activated by relu
    concat_xg = layers.add([phi_g, theta_x])
    act_xg = layers.Activation('relu')(concat_xg)

# Conduct ψ operation on act_xg, then activated by sigmoid
    psi = layers.Conv2D(1, (1, 1), padding='same')(act_xg)
    sigmoid_xg = layers.Activation('sigmoid')(psi)

# Conduct multiply operation on [upsample_psi, x]
    result = layers.multiply([sigmoid_xg, x])
    result_bn = layers.BatchNormalization()(result)
    
    return result_bn

In [11]:
# Definition of AM-SegNet with lightweight block and attention mechanism

def AM_SegNet(input_shape, num_classes, dropout, batch_norm):

    # parameters of network congfiguration

    filter_num = 12 # number of filters
    filter_size = 3 # size of filters
    up_samp_size = 2 # size of upsampling filters

    inputs = layers.Input(input_shape)

    # Downsampling

    # Downsampling step 1
    conv_1 = lw_conv_block(inputs, filter_size, 1*filter_num, dropout, batch_norm)
    pool_1 = layers.MaxPooling2D(pool_size=(2,2))(conv_1)

    # Downsampling step 2
    conv_2 = lw_conv_block(pool_1, filter_size, 2*filter_num, dropout, batch_norm)
    pool_2 = layers.MaxPooling2D(pool_size=(2,2))(conv_2)

    # Downsampling step 3
    conv_3 = lw_conv_block(pool_2, filter_size, 4*filter_num, dropout, batch_norm)
    pool_3 = layers.MaxPooling2D(pool_size=(2,2))(conv_3)

    # Downsampling step 5
    conv_4 = lw_conv_block(pool_3, filter_size, 8*filter_num, dropout, batch_norm)
    pool_4 = layers.MaxPooling2D(pool_size=(2,2))(conv_4)

    # Standard convolution only
    conv_5_1 = stand_conv_block(pool_4, filter_size, 64*filter_num, dropout, batch_norm)
    conv_5_2 = stand_conv_block(conv_5_1, filter_size, 64*filter_num, dropout, batch_norm)
    conv_5_3 = stand_conv_block(conv_5_2, filter_size, 64*filter_num, dropout, batch_norm)

    #Calculate Attention
    conv_att = attention_block(conv_5_1, conv_5_3, 64*filter_num)
    conv_5 = layers.add([conv_5_3, conv_att])

    # Upsampling

    # Upsampling step 1

    up_1 = layers.UpSampling2D(size=(up_samp_size, up_samp_size), data_format="channels_last")(conv_5)
    up_1 = layers.concatenate([up_1, conv_4], axis=3)
    up_conv_1 = stand_conv_block(up_1, filter_size, 16*filter_num, dropout, batch_norm)
   
    # Upsampling step 2
    up_conv_1 = layers.Conv2D(16*filter_num, (filter_size, filter_size), padding='same')(up_conv_1)
    up_2 = layers.UpSampling2D(size=(up_samp_size, up_samp_size), data_format="channels_last")(up_conv_1)
    up_2 = layers.concatenate([up_2, conv_3], axis=3)
    up_conv_2 = stand_conv_block(up_2, filter_size, 8*filter_num, dropout, batch_norm)

    # Upsampling step 3
    up_conv_2 = layers.Conv2D(8*filter_num, (filter_size, filter_size), padding='same')(up_conv_2)
    up_3 = layers.UpSampling2D(size=(up_samp_size, up_samp_size), data_format="channels_last")(up_conv_2)
    up_3 = layers.concatenate([up_3, conv_2], axis=3)
    up_conv_3 = stand_conv_block(up_3, filter_size, 4*filter_num, dropout, batch_norm)

    # Upsampling step 4
    up_conv_3 = layers.Conv2D(4*filter_num, (filter_size, filter_size), padding='same')(up_conv_3)
    up_4 = layers.UpSampling2D(size=(up_samp_size, up_samp_size), data_format="channels_last")(up_conv_3)
    up_4 = layers.concatenate([up_4, conv_1], axis=3)
    up_conv_4 = stand_conv_block(up_4, filter_size, 2*filter_num, dropout, batch_norm)

    # 1*1 convolutional layers
    conv_final = layers.Conv2D(num_classes, kernel_size=(1,1))(up_conv_4)
    conv_final = layers.BatchNormalization(axis=3)(conv_final)
    conv_final = layers.Activation('softmax')(conv_final)  #Change to softmax for multichannel
    
    # Model 
    model = models.Model(inputs, conv_final, name="AM-SegNet")
    
    # print model summary for details
    print(model.summary())

    return model

In [12]:
input_size_x= 256 
input_size_y= 512
input_size = (input_size_x,input_size_y,1)


class_num=5
fine_tune_lr = 1e-4

dropout=0.0

In [13]:
model = AM_SegNet(input_size, class_num, dropout, batch_norm=True)

model.load_weights(r'C:\Users\joser\Downloads\Model weight_AM_SegNet.hdf5')

Model: "AM-SegNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 512, 12  24          ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 activation (Activation)        (None, 256, 512, 12  0           ['conv2d[0][0]']                 
                                )                                                         

In [14]:
for layer in model.layers[:10]:  
    layer.trainable = False

for layer in model.layers[10:]:
    layer.trainable = True

In [15]:
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (input_size_y, input_size_x))
    img = np.expand_dims(img, axis=-1)
    return img

In [16]:
def preprocess_mask(mask_path):
    mask = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask,(input_size_y,input_size_x))
    mask = mask.astype("float32")
    mask[mask == 0] = 4 
    mask = mask - 1     
    mask = np.clip(mask, 0, 4)
    mask_onehot = to_categorical(mask, num_classes=class_num)
    mask_onehot = mask_onehot.reshape(mask.shape[0],mask.shape[1],class_num)
    return mask_onehot

In [ ]:
image_paths = ["path_to_image1.png", "path_to_image2.png", ...] 
mask_paths = ["path_to_mask1.png", "path_to_mask2.png", ...]

X_train = np.array([preprocess_image(img_path) for img_path in image_paths])
Y_train = np.array([preprocess_mask(mask_path) for mask_path in mask_paths])

X_train = X_train.reshape(-1, input_size_x, input_size_y, 1) 
Y_train = Y_train.reshape(-1, input_size_x, input_size_y, class_num)


In [18]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Keras version:", tf.keras.__version__)

TensorFlow version: 2.12.0
Keras version: 2.12.0


In [19]:
from keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

loss_fn = tf.keras.losses.CategoricalCrossentropy()

model.compile(
    optimizer=Adam(learning_rate=fine_tune_lr),
    loss=loss_fn,
    metrics=['accuracy']
)

In [22]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("fine_tuned_model.hdf5", monitor='loss', save_best_only=True, verbose=1)

batch_size = 8
epochs = 10

# Train the model
model.fit(
    X_train, Y_train,
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
    callbacks=[checkpoint]
)

Debugging due to label 0 of Matlab tool

In [16]:
unique_values = np.unique(mask)
print("Unique values in mask:", unique_values)

if np.any(unique_values >= classes):
    print("⚠️ Warning: Some pixel values exceed expected range!")

unique_values, counts = np.unique(mask, return_counts=True)
print("Unique values in mask:", unique_values)
print("Count of each value:", dict(zip(unique_values, counts)))

Unique values in mask: [0 1 2 3 4]
Unique values in mask: [0 1 2 3 4]
Count of each value: {0: 69942, 1: 13288, 2: 960142, 3: 192192, 4: 204436}
